In [1]:
import ffn 
from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import matplotlib.pyplot as plt
import tkinter
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
import pandas as pd
import seaborn as sns
import os
#os.getcwd()

C:\Users\rosorio\.conda\envs\StocksEnv\lib\site-packages\ffn\core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
C:\Users\rosorio\.conda\envs\StocksEnv\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\Users\rosorio\.conda\envs\StocksEnv\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
'''with open('return_std.txt', 'r') as file:
    data = file.read().replace('\n', ',')'''

"with open('return_std.txt', 'r') as file:\n    data = file.read().replace('\n', ',')"

In [4]:
#sp500 = ffn.get('spy', start='2007-01-01')
#sp500.index = sp500.index.to_period("Q")
#sp500.resample('Q').mean()

In [11]:
prices = ffn.get('VGT', start='2015-01-01')
fh = open('sectors.txt')
for line in fh:
    new_df = ffn.get(line, start='2015-01-01')
    prices = pd.concat([prices, new_df], axis=1)
fh.close()

In [13]:
benchmark = ffn.get('spy', start='2015-01-01')
msft = ffn.get('msft', start='2015-01-01')
prices=prices.rebase()
plt.show()

In [14]:
prices=prices.rebase()
prices=prices.dropna(axis='columns') #drop columns where at least one row is missing

In [ ]:
""" stats = prices[['bkng', 'amzn','googl', 'goog','azo', 'cmg','mtd', 'isrg']].calc_stats()
stats.display()
alpha, beta = alpha_beta(msft, benchmark)
print(beta)
prices.shape
prices.mean(axis=0, skipna = True)"""

In [23]:
prices=prices.loc[:, prices.mean() > 100] # keep columns that did not grow.
prices.to_csv(r'sectors.csv')

In [15]:
dias=prices.shape[0]
empresas=prices.shape[1]
prices.shape

(1390, 11)

In [16]:
returns = prices.pct_change() 
# mean daily return and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()
 
# portfolio weights
weights = np.full(empresas,1/empresas, dtype=float, order='F')
 
portfolio_return = round(np.sum(mean_daily_returns * weights) * dias,2)
portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(dias),2)

print("Expected annualised return: " + str(portfolio_return))
print("Volatility: " + str(portfolio_std_dev))

Expected annualised return: 0.46
Volatility: 0.44


In [17]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
#print(cleaned_weights)
ef.portfolio_performance(verbose=True)

Expected annual return: 22.3%
Annual volatility: 22.7%
Sharpe Ratio: 0.89


(0.22295809485480286, 0.22694302222296586, 0.8943129992135277)

In [18]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.lp_portfolio()
print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

10 out of 11 tickers were removed
{'vgt': 33}
Funds remaining: $224.33


In [19]:
pd.DataFrame(allocation.items(), columns=['Selected', 'number'])
prices_firmselected=prices[prices.columns.intersection(pd['Selected'])]

TypeError: 'module' object is not subscriptable

In [21]:
def correlation(equity_list): 
    # Get correlation and sort by sum
    sum_corr = equity_list.corr().sum().sort_values(ascending=True).index.values
    return equity_list[sum_corr].corr()

In [22]:
plt.figure(figsize=(13, 8))
sns.heatmap(correlation(prices_firmselected), annot=True, cmap='Greens')
plt.show()

NameError: name 'prices_firmselected' is not defined